# Excel Range 功能测试

本 notebook 用于测试 `excel_range` 函数的各种功能，包括：
- 单个单元格解析
- 区域范围解析
- 超出边界的自动裁剪
- 多区域合并
- header 和 index_col 参数

In [ ]:
import pandas as pd
import numpy as np
import sys
import warnings

# 导入 xlgrab
import xlgrab

# 显示所有警告
warnings.simplefilter('always')

print(f"xlgrab 版本: {xlgrab.__version__ if hasattr(xlgrab, '__version__') else 'unknown'}")
print(f"pandas 版本: {pd.__version__}")

## 1. 创建测试数据

创建一个模拟 Excel 数据的 DataFrame

In [ ]:
# 创建一个 10x6 的测试 DataFrame，模拟 Excel 数据
df = pd.DataFrame({
    'A': ['姓名', '张三', '李四', '王五', '赵六', '钱七', '孙八', '周九', '吴十', '郑十一'],
    'B': ['年龄', 25, 30, 28, 35, 22, 27, 31, 29, 26],
    'C': ['部门', '销售', '技术', '销售', '技术', '市场', '销售', '技术', '市场', '销售'],
    'D': ['工资', 8000, 12000, 9000, 15000, 7000, 8500, 13000, 7500, 9500],
    'E': ['城市', '北京', '上海', '北京', '上海', '广州', '深圳', '上海', '广州', '北京'],
    'F': ['入职日期', '2020-01', '2019-05', '2021-03', '2018-08', '2022-01', '2020-11', '2019-02', '2021-09', '2020-06']
})

print("原始测试数据 (10行6列):")
print(df)
print(f"\nDataFrame 形状: {df.shape}")

## 2. 测试单个单元格解析

In [ ]:
print("测试 1: 获取单个单元格 'B2'")
result = df.excel_range('B2', header=False)
print(result)
print(f"结果形状: {result.shape}")

In [ ]:
print("测试 2: 获取单个单元格 'D5'")
result = df.excel_range('D5', header=False)
print(result)
print(f"结果形状: {result.shape}")

## 3. 测试正常区域范围

In [ ]:
print("测试 3: 获取区域 'A1:C4' (不使用 header)")
result = df.excel_range('A1:C4', header=False)
print(result)
print(f"结果形状: {result.shape}")

In [ ]:
print("测试 4: 获取区域 'A1:C4' (使用第一行作为 header)")
result = df.excel_range('A1:C4', header=True)
print(result)
print(f"结果形状: {result.shape}")

In [ ]:
print("测试 5: 获取区域 'B2:E6' (包含数据部分)")
result = df.excel_range('B2:E6', header=False)
print(result)
print(f"结果形状: {result.shape}")

## 4. 测试 header 和 index_col 参数

In [ ]:
print("测试 6: 使用第一行作为列名，第一列作为索引")
result = df.excel_range('A1:D6', header=True, index_col=0)
print(result)
print(f"结果形状: {result.shape}")
print(f"索引名称: {result.index.name}")
print(f"列名: {list(result.columns)}")

In [ ]:
print("测试 7: 使用列名指定索引列")
result = df.excel_range('A1:D6', header=True, index_col='姓名')
print(result)
print(f"结果形状: {result.shape}")

## 5. 测试超出边界的自动裁剪

这是新增的功能：当请求的区域超出 DataFrame 范围时，自动裁剪到有效边界

In [ ]:
print("测试 8: 超出行范围 'B2:D20' (DataFrame 只有 10 行)")
print("预期：自动裁剪到第 10 行，并显示警告")
result = df.excel_range('B2:D20', header=False)
print(result)
print(f"结果形状: {result.shape}")
print(f"实际返回行数: {len(result)}")

In [ ]:
print("测试 9: 超出列范围 'A1:Z5' (DataFrame 只有 6 列 A-F)")
print("预期：自动裁剪到第 F 列，并显示警告")
result = df.excel_range('A1:Z5', header=False)
print(result)
print(f"结果形状: {result.shape}")
print(f"实际返回列数: {len(result.columns)}")

In [ ]:
print("测试 10: 同时超出行和列范围 'C3:Z100'")
print("预期：自动裁剪到 DataFrame 的实际边界")
result = df.excel_range('C3:Z100', header=False)
print(result)
print(f"结果形状: {result.shape}")

## 6. 测试多区域合并

In [ ]:
print("测试 11: 合并多个区域 'A2:B4' 和 'D2:E4'")
result = df.excel_range('A2:B4', 'D2:E4', header=False)
print(result)
print(f"结果形状: {result.shape}")

In [ ]:
print("测试 12: 使用逗号分隔的多区域字符串 'A2:B4,D2:E4,F2:F4'")
result = df.excel_range('A2:B4,D2:E4,F2:F4', header=False)
print(result)
print(f"结果形状: {result.shape}")

## 7. 测试边界情况

In [ ]:
print("测试 13: 获取整个 DataFrame 'A1:F10'")
result = df.excel_range('A1:F10', header=False)
print(result)
print(f"结果形状: {result.shape}")
print(f"是否与原始 DataFrame 相同: {result.equals(df)}")

In [ ]:
print("测试 14: 获取最后一个单元格 'F10'")
result = df.excel_range('F10', header=False)
print(result)
print(f"结果形状: {result.shape}")

In [ ]:
print("测试 15: 获取第一个单元格 'A1'")
result = df.excel_range('A1', header=False)
print(result)
print(f"结果形状: {result.shape}")

## 8. 错误处理测试

In [ ]:
print("测试 16: 起始位置超出范围（应该报错）")
try:
    result = df.excel_range('Z50:Z60', header=False)
    print("错误：应该抛出异常但没有！")
except ValueError as e:
    print(f"✓ 正确捕获异常: {e}")

In [ ]:
print("测试 17: 无效的区域格式（应该报错）")
try:
    result = df.excel_range('ABC', header=False)
    print("错误：应该抛出异常但没有！")
except (ValueError, Exception) as e:
    print(f"✓ 正确捕获异常: {e}")

## 9. 实际应用场景示例

In [ ]:
print("场景 1: 从 Excel 表格中提取员工信息（带表头）")
employees = df.excel_range('A1:E6', header=True, index_col='姓名')
print(employees)
print(f"\n员工数量: {len(employees)}")
print(f"平均年龄: {employees['年龄'].mean():.1f}")
print(f"平均工资: {employees['工资'].mean():.0f}")

In [ ]:
print("场景 2: 提取特定部门的工资数据")
salary_data = df.excel_range('C2:D10', header=False)
salary_data.columns = ['部门', '工资']
print(salary_data)
print(f"\n按部门统计平均工资:")
print(salary_data.groupby('部门')['工资'].mean().sort_values(ascending=False))

In [ ]:
print("场景 3: 提取多个不连续的数据区域并合并")
# 提取姓名、工资和城市三列（不连续）
selected_data = df.excel_range('A1:A6', 'D1:D6', 'E1:E6', header=True)
print(selected_data)
print(f"\n按城市统计平均工资:")
print(selected_data.groupby('城市')['工资'].mean().sort_values(ascending=False))

## 10. 性能测试（可选）

In [ ]:
import time

# 创建一个更大的 DataFrame
large_df = pd.DataFrame(
    np.random.randn(1000, 26),
    columns=[chr(65+i) for i in range(26)]  # A-Z
)

print(f"大型 DataFrame 形状: {large_df.shape}")

# 测试性能
start = time.time()
result = large_df.excel_range('B10:Y500', header=False)
elapsed = time.time() - start

print(f"\n提取区域 'B10:Y500' 耗时: {elapsed*1000:.2f} ms")
print(f"结果形状: {result.shape}")

## 总结

### 测试通过的功能：
1. ✓ 单个单元格解析
2. ✓ 区域范围解析
3. ✓ header 参数（使用第一行作为列名）
4. ✓ index_col 参数（指定索引列）
5. ✓ 超出边界自动裁剪（新功能）
6. ✓ 多区域合并
7. ✓ 错误处理

### 新增功能亮点：
- **单个单元格支持**：可以直接使用 `'B2'` 而不需要 `'B2:B2'`
- **智能边界裁剪**：超出范围时自动裁剪到有效边界，而不是报错
- **友好的警告信息**：清楚地告知用户发生了裁剪